In [1]:
import sys
sys.path.append("../")
import numpy as np 
from utils import *
from lattice import *
import pandas
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensornetwork as tn

In [2]:
df = get_seed_and_loss("../../../worm_result")

In [14]:
sps = 3
bd = 2
res = {}
for seed in range(0, 11):
    parent_dir = f"../array/torch/FF1D_loc/s_3_r_2_lt_1_d_1_seed_{seed}/original_mel_LION/lr_0.005_epoch_20000"
    u_path, lv = path_with_lowest_loss(parent_dir)
    A = FF.block1D(bd, sps, bd, seed=seed)
    # mps_nodes, mps_edges = FF.create_MPS(6, A)
    # for k in range(len(mps_edges)):
    #     C = tn.contract(mps_edges[k])
    # y = C.tensor.reshape(sps**2, -1)
    A2 = np.einsum("ijk,klm->jlim", A, A).reshape(sps**2, bd**2)
    U, s , V = np.linalg.svd(A2)
    Up = U[:, len(s):]
    h = Up @ Up.T
    L = 6
    H = sum_ham(h, [[i, (i + 1) % L] for i in range(L)], L, sps)
    res[seed] = {
        "E" : np.linalg.eigvalsh(H),
        "loss" : lv
    }


In [27]:
for key, val in res.items():
    E = val["E"]
    gap = E[1] - E[0]
    res[key]["gap"]  = gap
    print(val["loss"], gap)

0.17387 0.014409947267715335
0.0 3.808084226575378e-08
2e-05 3.088894926684373e-08
0.26422 0.022424657326068447
0.09499 0.012584317829217564
0.06638 0.00510515577127765
0.027 0.00046814891419191395
0.00521 0.0012275567273297924
0.0005 0.00025582230076526486
0.11884 0.23563248342923143
0.15375 0.013735076408410358


## Calculate transfer matrix

In [36]:
for seed in range(0, 11):
    A = FF.block1D(bd, sps, bd, seed=seed)
    A_ = A.transpose(1,0,2)
    A_tilde_ = np.einsum("ijk,ilm->jlkm", A_, A_)
    A_tilde = A_tilde_.reshape(4,4)
    e_t = np.linalg.eigvalsh(A_tilde)
    res[seed]["et"]  = e_t
    
    A2 = np.einsum("ijk,klm->jlim",A,A)
    A2_ = A2.reshape(sps**2, bd, bd)
    A2_tilde_ = np.einsum("ijk,ilm->jlkm", A2_, A2_)
    A2_tilde = A2_tilde_.reshape(4,4)
    e_t = np.linalg.eigvalsh(A2_tilde)
    res[seed]["et2"]  = e_t

In [37]:
for key, val in res.items():
    print(f'loss: {val["loss"]}')
    print(f'gap : {val["gap"]}')
    print(f"et : {val['et']}")
    print(f"et2 : {val['et2']}")
    print("\n")

loss: 0.17387
gap : 0.014409947267715335
et : [1.07903285 1.41623274 4.14088113 8.92945925]
et2 : [ 1.16431189  2.00571517 17.14689655 79.73524257]


loss: 0.0
gap : 3.808084226575378e-08
et : [-3.52683104 -2.4892931   3.46655985  5.3265312 ]
et2 : [ 6.19658016 12.01703717 12.43853718 28.37193457]


loss: 2e-05
gap : 3.088894926684373e-08
et : [-1.80583275 -1.69885453  7.63632091  8.15280263]
et2 : [ 2.8861067   3.26103194 58.31339699 66.46819078]


loss: 0.26422
gap : 0.022424657326068447
et : [-3.0497284  -0.4825839   1.14272441  4.25264852]
et2 : [ 0.23288722  1.30581908  9.30084333 18.08501945]


loss: 0.09499
gap : 0.012584317829217564
et : [0.7080587  0.83183422 0.92752041 2.80439502]
et2 : [0.50134713 0.69194816 0.86029412 7.86463145]


loss: 0.06638
gap : 0.00510515577127765
et : [-1.50881426 -0.8815801   0.66147717  6.17440182]
et2 : [ 0.43755205  0.77718348  2.27652046 38.12323784]


loss: 0.027
gap : 0.00046814891419191395
et : [-5.25731758 -2.88693889  2.25853235 12.8175936